In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


In [3]:
heart_data = pd.read_csv('heart.csv')
heart_data.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
heart_data.shape

(303, 14)

In [5]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
heart_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [7]:
heart_data['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

In [8]:
heart_data.shape

(303, 14)

In [9]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [10]:
x = heart_data.iloc[:,:-1].values
print(x)

[[63.  1.  3. ...  0.  0.  1.]
 [37.  1.  2. ...  0.  0.  2.]
 [41.  0.  1. ...  2.  0.  2.]
 ...
 [68.  1.  0. ...  1.  2.  3.]
 [57.  1.  0. ...  1.  1.  3.]
 [57.  0.  1. ...  1.  1.  2.]]


In [11]:
y = heart_data.iloc[:,-1].values
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


# Model Selection

**Comparing the models with default hyperparameter values using cross validation**

In [12]:
# list of models

models = [LogisticRegression(max_iter=1000),SVC(kernel='linear'),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

# max_iter to avoid warnning (discussion in pipeline)
# kernel is esential 
# random_state to avoid the differnt value for each type run



In [13]:
def compare_model_cross_validation():
    for model in models:
        
        cv_score = cross_val_score(model,x,y,cv=5)   #to train,test and getting accuracy of dataset for list[models]
                                                     # we use function cross_val_score(model=index of list,x,y,cv=5 means to 
                                                    # train model 5 times with different order of elements ).
       
        mean_accuracy = sum(cv_score)/len(cv_score) # as model trained 5 times so calulate mean accuracy
        mean_accuracy = mean_accuracy*100           # as accuracy is like 0.899987 so conver into 89.9987
        mean_accuracy = round(mean_accuracy,2)      # just want upto 2 decimals like 89.99
        
        print('Cross Validation accuracies for the model',model,'=',cv_score)
        print('Accuracy score of the ',model,'=',mean_accuracy,'%')
        print('----------------------------------------------------------------')
        
        

In [14]:
#calling the function

compare_model_cross_validation()

Cross Validation accuracies for the model LogisticRegression(max_iter=1000) = [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy score of the  LogisticRegression(max_iter=1000) = 82.83 %
----------------------------------------------------------------
Cross Validation accuracies for the model SVC(kernel='linear') = [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy score of the  SVC(kernel='linear') = 82.83 %
----------------------------------------------------------------
Cross Validation accuracies for the model KNeighborsClassifier() = [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy score of the  KNeighborsClassifier() = 64.39 %
----------------------------------------------------------------


C:\Users\malik bros\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\malik bros\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Se

Cross Validation accuracies for the model RandomForestClassifier(random_state=0) = [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Accuracy score of the  RandomForestClassifier(random_state=0) = 83.81 %
----------------------------------------------------------------


**comparing the models by using different hyperparameters using GridSearchCV**

In [15]:
# list of models

models_list = [LogisticRegression(max_iter=10000),SVC(),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

# max_iter to avoid warnning (discussion in pipeline)

# random_state to avoid the differnt value for each type run



In [16]:
# creating a dictionary that contains hyperparameters values for the models

model_hyperparameter = {
    
    'log_reg_p': {
        'c': [1,5,10,20]
    },
    'svc_p' : {
        'kernel': ['linear','poly','rbf','sigmiod'],
         'c' : [1,5,10,20]
    },
    
    'KNN_p': {
        
        'n_neighbor': [3,5,10]
    },
    
    'rand_for_cla_p': {
        'n_estimaters': [10,20,50,100]
    }
}

In [17]:
type(model_hyperparameter)

dict

In [18]:
print(model_hyperparameter.keys())

dict_keys(['log_reg_p', 'svc_p', 'KNN_p', 'rand_for_cla_p'])


In [19]:
model_keys = list(model_hyperparameter.keys())
print(model_keys)

['log_reg_p', 'svc_p', 'KNN_p', 'rand_for_cla_p']


In [20]:
# Appling GridSearchCV

def modelselection(list_of_models,hyperparameter_dic):
    result = []
    
    i=0
    
    for model in list_of_models:
        key = model_keys[i]
         
        params = hyperparameter_dic[key]
         
        i += 1
         
        print(model)
        print(params)   
        print('----------------------------')
        
        classifier = GridSearchCV(model,params,cv=5)
        classifier.fit(x,y)
        
        result.append({
            'model used': model,
            'highest score': classifier.best_score,
            'best hyperparameters': classifier.best_para_
            
        })
    result_dataframe = pd.DataFrame(result,columns = ['model used','highest score','best hyperparameters'])
    return result_dataframe

In [21]:
LogisticRegression.get_params().keys()

TypeError: get_params() missing 1 required positional argument: 'self'

In [22]:
modelselection(models_list,model_hyperparameter)

LogisticRegression(max_iter=10000)
{'c': [1, 5, 10, 20]}
----------------------------


ValueError: Invalid parameter c for estimator LogisticRegression(max_iter=10000). Check the list of available parameters with `estimator.get_params().keys()`.